# MLP Model

This simple model is used to verify that the supporting data loading, splitting, training and validation functions are working correctly by training the model on a small subset of the data. The model metrics should show distinct decreasing loss and attain a high training accuracy.

### Imports

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import ToTensor
from utils import split_dataset, train_model, plot_metrics, save_model

### Initialize Dataset

In [ ]:
# Constants
TRAIN_DATA_DIR = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train'
SAVE_MODEL_FILE = 'models/mlp.pt'
SEED = 0
NUM_SAMPLES = 87000
TRAIN_SPLIT = 0.8
BATCH_SIZE = 100
LEARN_RATE = 5e-4
EPOCHS = 10

In [ ]:
# Seed PyTorch
torch.manual_seed(SEED)

# Initialize dataset
dataset = torchvision.datasets.ImageFolder(
    root=TRAIN_DATA_DIR,
    transform=ToTensor()
)
num_inputs = np.array(dataset[0][0].numpy().shape).prod()
num_outputs = len(dataset.classes)

# Split dataset into training and validation sets
train_loader, valid_loader = split_dataset(dataset, NUM_SAMPLES, TRAIN_SPLIT, BATCH_SIZE, SEED)

# Check for CUDA GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

### Model

In [ ]:
# Multilayer perceptron
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden1 = nn.Linear(num_inputs, 1000)
        self.hidden2 = nn.Linear(1000, 200)
        self.output = nn.Linear(200, num_outputs)

    def forward(self, x):
        x = self.hidden1(x)
        x = F.relu(x)
        x = self.hidden2(x)
        x = F.relu(x)
        x = self.output(x)
        return x

# Instantiate model and move to GPU if available
model = MLP()
model = model.to(device)

# Print model details
from torchsummary import summary
summary(model, input_size=(num_inputs, ))

### Training

In [ ]:
metrics = train_model(model, train_loader, valid_loader, LEARN_RATE, EPOCHS, device)

In [ ]:
save_model(SAVE_MODEL_FILE, model, metrics)

### Evaluation

In [ ]:
plot_metrics(metrics)